Initializing Libraries and Global Variables

In [312]:
import pickle
import pandas as pd
import psycopg2
import os
import itertools

In [313]:
path = r'D:\\Projects\\ipums\\garrard_test\\'
dataname = r'ky_garrard_1870_88v_test_usa_00006.dat'
cbkname = r'ky_1870_cbk.pickle'
prefix = 'cn1870_'
table_name = 'census_1870'
cbkfile = path + cbkname
datafile = path + dataname

Pickle Functions

In [314]:
#Pickle functions
def load_pickle(filename, objectname):
        with open(path+filename, "rb") as infile:
            objectname = pickle.load(infile)
        print('Loaded from:', filename)

def save_pickle(filename, objectname):
    with open(path+filename, "wb") as outfile:
        pickle.dump(objectname, outfile)
    print('Saved to:',filename)

Loading the Codebook and Creating the Slice List

In [315]:
cbk = pd.read_pickle(cbkfile)
x = cbk['length'].astype('str')
slice_list = cbk['slice_obj'].tolist()
print('slice_list:\n',slice_list[0:10])

slice_list:
 [slice(0, 4, None), slice(4, 12, None), slice(12, 14, None), slice(14, 16, None), slice(16, 20, None), slice(20, 21, None), slice(21, 23, None), slice(23, 25, None), slice(25, 27, None), slice(27, 31, None)]


Database Functions (psycog2)

In [316]:
def open_database(database):
    conn = psycopg2.connect(dbname=database, user='postgres', password = '3701', host='localhost', port='5432')
    return conn

def create_cursor():
    cursor = conn.cursor()
    return cursor

def set_census_schema():
    cursor.execute("SET SCHEMA 'census'")

def close_database():
    conn.commit() #insert rollback logic
    conn.close()   

Data Processing Functions

In [317]:
def process_line(line, slices):
    limit = len(line)
    data = []
    for i in slices:
        len_slice = len(line[i])
        begin = limit
        limit = limit - len_slice
        if limit <= 0: break
        data.append(line[i])
    return data

def insert_row(tablename, ph, row):
    query = r"INSERT INTO " + tablename + " VALUES (" + ph + ") "
    print('executing the insert')
    cursor.execute(query, row)
    print('committing the record')
    #rs = cursor.fetchall()
    conn.commit()
    #for r in rs:
        #print(r)
    #cursor.execute(query, (data,)) # notice the comma after the tuple

def clear_table(tablename):
    clear = 'DELETE FROM ' + tablename

In [318]:
#cursor.execute("ROLLBACK")

Open the Database

In [319]:
conn = open_database('test')
cursor = create_cursor()
set_census_schema()

Process the Datafile (line by line) and Upload it to the Database

In [320]:
#input sequence
#filename - first a scalar, then a list to process more (* in a directory, or list of filenames)
#input (
#p = process from zero (set rec_count=1) - are you sure? This will wipe out resume data (y = proceed, anything else = repeat input)
#r = read json for filename (revert to p if not available), resume from (processed_count + 1) - "are you sure?"
#x = break
#else = repeat input loop)

In [321]:
# set up by the input sequence
rec_count = 1
processed_count = 0
records = 1
end_range = processed_count + 1 + records

# now proceeding to process the records
with open(datafile) as f: 
    while (True):
        line = f.readline() # this pulls the next line.
        if line == "": break
        if (rec_count >= end_range and records != 0): break
        if (rec_count) <= processed_count:
            print('skipping record',rec_count)
            rec_count += 1
            continue
        print ('processing record',rec_count)
        data = process_line(line, slice_list)
        print(type(data),data)
        total_fields = (len(data))
        ph = r'%s, '*(total_fields-1) + '%s'
        insert_row(table_name, ph, data)
        rec_count += 1
print('Total fields:',total_fields)
print('Start record:',processed_count+1)
print('End record:',rec_count-1)
print('Total records processed:', (rec_count - processed_count - 1))
#save rec_count as processed_count in the json file

processing record 1
<class 'list'> ['1870', '02095398', '06', '50', '0006', '1', '32', '51', '21', '0810', '2100810', '1', '000', '0000', '0000', '0000000', '01', '1', '0000000', '0001597', '840', '1', '0', '000', '00', '2', '01', '1', '1', '1', '0', '00073260', '0006', 'A9AEFCA0-B1B4-48F5-AF1E-88D9FF8B6460', '0001', '01', '06', '00', '0', '0', '00', '0', '0', '02', '1', '4', '0', '0', '26', '18', '01', '0101', '01', '1', '050', '99', '00', '1820', '1', '1', '100', '0', '000', '021', '02100', '1', '3', '0', '1', '1', '2', '0005', '100', '105', '00000400', '00000245', '03', 'A9AEFCA0-B1B4-48F5-AF1E-88D9FF8B6460', '01', 'qkNIx5kHEM6Up-GCglGJH', '1', '1', '1', '0', '0', '0', '0', '3.0-1.1']
executing the insert
committing the record
Total fields: 88
Start record: 1
End record: 1
Total records processed: 1


Commit and Close


In [322]:
close_database()

Inserting the tuples as a single value  

query = """  
    insert into t values %s  
    returning *  
"""
my_tuple = (2, 'b')  

cursor.execute(query, (my_tuple,)) # Notice the comma after my_tuple  
rs = cursor.fetchall()  
conn.commit()  
for row in rs:  
    print row  